In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData

from data.node_datamodule import NodeDataModule
from models.graph_embedding import GraphEmbedding

In [3]:
def adata2data(adata:AnnData)-> Data:
    sq.gr.spatial_neighbors(adata, coord_type="generic")
    r, c = adata.obsp['spatial_connectivities'].nonzero()
    edge_index = torch.vstack([torch.from_numpy(r).to(
        torch.long), torch.from_numpy(c).to(torch.long)])
    cell_type = torch.from_numpy(pd.get_dummies(adata.obs).to_numpy())
    gene_expression = torch.from_numpy(adata.X)
    return Data(
        edge_index=edge_index,
        x=gene_expression,
        y=cell_type
    )

In [4]:
adata = sq.datasets.imc()


In [5]:
dm = NodeDataModule(adata=adata,adata2data_fn=adata2data, num_workers = 12)

In [6]:
model = GraphEmbedding(num_features=34,latent_dim=30,lr=0.0001,weight_decay=0.000001)

In [7]:
trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=1000,log_every_n_steps=1)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
trainer.fit(model,datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params
------------------------------------
0 | model   | GraphAE | 15.5 K
1 | loss_fn | MSELoss | 0     
------------------------------------
15.5 K    Trainable params
0         Non-trainable params
15.5 K    Total params
0.062     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Exception ignored in: <function _releaseLock at 0x7f67dca69430>
Traceback (most recent call last):
  File "/home/sel/anaconda3/envs/ncem/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


RuntimeError: DataLoader worker (pid(s) 24244, 24256, 24268, 24281, 24293, 24305) exited unexpectedly